In [1]:
import pandas as pd 
import numpy as np

In [2]:
weather_data = pd.read_csv('./data/weather_data4.csv')
bike_data = pd.read_csv('./data/combined_csv.csv')

In [3]:
weather_data.head()

,date,time,weather_description,temp_in_f,humidity_in_%,pressure,precipitation_in_inches,visibility_in_miles,wind_direction,wind_speed_in_mph,temp_in_f_delta,pressure_delta,humidity_delta,visibility_delta,precipitation_delta,wind_speed_delta
0,2017-12-31,00:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.0,N,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-12-31,01:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.0,N,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2017-12-31,02:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.3,N,0.0,0.0,0.0,0.0,0.3,0.0,0.0
3,2017-12-31,03:00:00,Mist and Fog,32.0,100.0,30.0,0.0,4.3,N,0.0,0.0,0.0,0.0,0.3,0.0,0.0
4,2017-12-31,04:00:00,Clear,33.0,100.0,30.0,0.0,9.0,N,0.0,1.0,0.0,0.0,5.0,0.0,0.0


In [4]:
bike_data.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,552,2018-01-01 00:05:06,2018-01-01 00:14:18,31104,Adams Mill & Columbia Rd NW,31400,Georgia & New Hampshire Ave NW,W00886,Member
1,1282,2018-01-01 00:14:30,2018-01-01 00:35:53,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W01435,Casual
2,1265,2018-01-01 00:14:53,2018-01-01 00:35:58,31321,15th St & Constitution Ave NW,31321,15th St & Constitution Ave NW,W21242,Casual
3,578,2018-01-01 00:15:31,2018-01-01 00:25:09,31406,14th & Upshur St NW,31103,16th & Harvard St NW,W21322,Casual
4,372,2018-01-01 00:18:02,2018-01-01 00:24:15,31618,4th & East Capitol St NE,31619,Lincoln Park / 13th & East Capitol St NE,W00119,Member


In [5]:
weather_data['time'] = weather_data['time'].astype('str') 
weather_data['date'] = weather_data['date'].astype('str') 
weather_data.dtypes

date                        object
time                        object
weather_description         object
temp_in_f                  float64
humidity_in_%              float64
pressure                   float64
precipitation_in_inches    float64
visibility_in_miles        float64
wind_direction              object
wind_speed_in_mph          float64
temp_in_f_delta            float64
pressure_delta             float64
humidity_delta             float64
visibility_delta           float64
precipitation_delta        float64
wind_speed_delta           float64
dtype: object

## Cleaning the data

In [6]:
## convert to datetime 
bike_data['Start date'] = pd.to_datetime(bike_data['Start date'])
bike_data['End date'] = pd.to_datetime(bike_data['End date'])

In [7]:
## Separating date and time
bike_data['start_date'] = bike_data['Start date'].dt.date
bike_data['end_date'] = bike_data['End date'].dt.date
bike_data['start_time'] = bike_data['Start date'].dt.time
bike_data['end_time'] = bike_data['End date'].dt.time

In [8]:
# get duration in minutes 
bike_data['duration_in_mins'] = (bike_data['End date'] - bike_data['Start date']) / np.timedelta64(1,'m') 

In [9]:
bike_data = bike_data.rename(columns={'Start station number': 'start_station_id', 'Start station': 'start_station', 
                         'End station number': 'end_station_id', 'End station':'end_station', 
                         'Bike number': 'bike_id', 'Member type': 'member_type'})

## Sanity checks 

In [10]:
# checking number of nulls 
weather_data['time'].value_counts()

20:00:00    365
12:00:00    365
14:00:00    365
05:00:00    365
15:00:00    365
07:00:00    365
21:00:00    365
03:00:00    365
22:00:00    365
10:00:00    365
16:00:00    365
18:00:00    365
06:00:00    365
17:00:00    365
08:00:00    365
19:00:00    365
11:00:00    365
04:00:00    365
09:00:00    365
13:00:00    365
23:00:00    364
01:00:00    364
02:00:00    363
00:00:00    363
nan           2
Name: time, dtype: int64

## Merge weather data and bike data

In [11]:
# get nearest hour
bike_data['start_time'] = bike_data['start_time'].apply(lambda x:x.strftime('%H') + ':00:00')
bike_data['end_time'] = bike_data['end_time'].apply(lambda x:x.strftime('%H') + ':00:00')

In [12]:
bike_data.tail()

,Duration,Start date,End date,start_station_id,start_station,end_station_id,end_station,bike_id,member_type,start_date,end_date,start_time,end_time,duration_in_mins
3542679,3313,2018-12-31 23:37:22,2019-01-01 00:32:36,31615,6th & H St NE,31624,North Capitol St & F St NW,W00159,Member,2018-12-31,2019-01-01,23:00:00,00:00:00,55.233333
3542680,888,2018-12-31 23:42:18,2018-12-31 23:57:07,31618,4th & East Capitol St NE,31509,New Jersey Ave & R St NW,W22026,Member,2018-12-31,2018-12-31,23:00:00,23:00:00,14.816667
3542681,2332,2018-12-31 23:44:23,2019-01-01 00:23:16,31907,Franklin & S Washington St,32406,Fleet St & Waterfront St,W23900,Member,2018-12-31,2019-01-01,23:00:00,00:00:00,38.883333
3542682,307,2018-12-31 23:52:42,2018-12-31 23:57:50,31114,18th St & Wyoming Ave NW,31103,16th & Harvard St NW,W00761,Member,2018-12-31,2018-12-31,23:00:00,23:00:00,5.133333
3542683,1062,2018-12-31 23:54:09,2019-01-01 00:11:52,31110,20th St & Florida Ave NW,31014,Lynn & 19th St North,W21324,Member,2018-12-31,2019-01-01,23:00:00,00:00:00,17.716667


In [13]:
bike_data = bike_data.drop(['Start date', 'End date', 'Duration', 'bike_id', 'end_station', 'start_station'], axis = 1)

In [14]:
ins = bike_data[['start_date', 'start_time', 'start_station_id', 'member_type']]
outs = bike_data[['end_date', 'end_time', 'end_station_id', 'member_type']]

In [15]:
ins.shape

(3542684, 4)

In [16]:
outs.shape

(3542684, 4)

In [17]:
ins = ins.groupby(['start_date', 'start_time', 'start_station_id', 'member_type']).size().reset_index(name='total_out')

In [18]:
outs = outs.groupby(['end_date', 'end_time', 'end_station_id', 'member_type']).size().reset_index(name='total_in')

In [19]:
ins = ins.rename(columns={"start_date": "date", "start_time": "time", 'start_station_id':'station'})
outs = outs.rename(columns={"end_date": "date", "end_time": "time", 'end_station_id':'station'})

In [20]:
outs.head()

,date,time,station,member_type,total_in
0,2018-01-01,00:00:00,31045,Member,3
1,2018-01-01,00:00:00,31079,Member,1
2,2018-01-01,00:00:00,31098,Member,1
3,2018-01-01,00:00:00,31101,Member,1
4,2018-01-01,00:00:00,31103,Casual,1


In [21]:
final = pd.merge(ins, outs, on=['date', 'time', 'station', 'member_type'], how = 'outer')

In [22]:
final.isna().sum()

date                0
time                0
station             0
member_type         0
total_out      578762
total_in       597369
dtype: int64

In [23]:
final = final.fillna(0)

In [24]:
# drop days where there are null python 
print(weather_data.isna().sum())
weather_data.loc[weather_data['visibility_delta'].isna()]['date']

date                        0
time                        0
weather_description         7
temp_in_f                   1
humidity_in_%               1
pressure                    2
precipitation_in_inches     1
visibility_in_miles        11
wind_direction              0
wind_speed_in_mph           2
temp_in_f_delta             3
pressure_delta              4
humidity_delta              3
visibility_delta           15
precipitation_delta         3
wind_speed_delta            5
dtype: int64


0       2017-12-31
5879    2018-09-02
5880    2018-09-02
5881    2018-09-02
5882    2018-09-02
5883    2018-09-02
5884    2018-09-02
5885    2018-09-02
5886    2018-09-02
5887    2018-09-02
5888    2018-09-02
7673    2018-11-15
7674           nan
7675           nan
7676    2018-11-17
Name: date, dtype: object

In [25]:
weather_data.loc[(weather_data['date'] == '2018-09-02') & (weather_data['visibility_in_miles'].isnull()), 'visibility_in_miles'] = 10
weather_data.loc[(weather_data.date == '2018-09-02') & (weather_data.visibility_delta.isnull()), 'visibility_delta'] = 0
weather_data.loc[(weather_data.date == '2018-09-02')& (weather_data.weather_description.isnull()), 'weather_description'] = 'Cloudy'

In [26]:
weather_data.loc[(weather_data['date'] == '2018-11-15') & (weather_data['visibility_in_miles'].isnull())
                 ,'visibility_in_miles'] = 2.0
weather_data.loc[(weather_data['date'] == '2018-11-15') & (weather_data['wind_speed_in_mph'].isnull())
                 ,'wind_speed_in_mph'] = 0.0     
weather_data.loc[(weather_data['date'] == '2018-11-15') & (weather_data['visibility_delta'].isnull())
                 ,'visibility_delta'] = 0.0
weather_data.loc[(weather_data['date'] == '2018-11-15') & (weather_data['wind_speed_delta'].isnull())
                 ,'wind_speed_delta'] = 0.0
weather_data.loc[(weather_data['date'] == '2018-11-15') & (weather_data['weather_description'].isnull())
                 ,'weather_description'] = 'Cloudy'

weather_data.loc[(weather_data['date'] == '2018-11-17') & (weather_data['temp_in_f_delta'].isnull())
                 ,'temp_in_f_delta'] = 2.0
weather_data.loc[(weather_data['date'] == '2018-11-17') & (weather_data['pressure_delta'].isnull())
                 ,'pressure_delta'] = 0.0     
weather_data.loc[(weather_data['date'] == '2018-11-17') & (weather_data['visibility_delta'].isnull())
                 ,'visibility_delta'] = 0.0
weather_data.loc[(weather_data['date'] == '2018-11-17') & (weather_data['wind_speed_delta'].isnull())
                 ,'wind_speed_delta'] = 0.0
weather_data.loc[(weather_data['date'] == '2018-11-17') & (weather_data['humidity_delta'].isnull())
                 ,'humidity_delta'] = 0.0
weather_data.loc[(weather_data['date'] == '2018-11-17') & (weather_data['precipitation_delta'].isnull())
                 ,'precipitation_delta'] = 0.0

In [27]:
weather_data = weather_data.drop(['precipitation_delta', 'precipitation_in_inches'], axis = 1)

In [28]:
weather_data.tail()

,date,time,weather_description,temp_in_f,humidity_in_%,pressure,visibility_in_miles,wind_direction,wind_speed_in_mph,temp_in_f_delta,pressure_delta,humidity_delta,visibility_delta,wind_speed_delta
8751,2018-12-31,19:00:00,Cloudy,49.0,96.0,29.0,10.0,WSW,6.3,5.0,0.0,-4.0,4.0,4.6
8752,2018-12-31,20:00:00,Mostly Cloudy,52.0,94.0,29.0,10.0,W,6.7,3.0,0.0,-2.0,0.0,0.4
8753,2018-12-31,21:00:00,Mostly Cloudy,53.0,98.0,29.0,10.0,W,6.7,1.0,0.0,4.0,0.0,0.0
8754,2018-12-31,22:00:00,Cloudy,54.0,96.0,29.0,10.0,W,5.7,1.0,0.0,-2.0,0.0,-1.0
8755,2018-12-31,23:00:00,Partly Cloudy,60.0,82.0,29.0,10.0,SSW,11.0,6.0,0.0,-14.0,0.0,5.3


In [29]:
final.dtypes

date            object
time            object
station          int64
member_type     object
total_out      float64
total_in       float64
dtype: object

In [30]:
final.head()

,date,time,station,member_type,total_out,total_in
0,2018-01-01,00:00:00,31019,Member,1.0,0.0
1,2018-01-01,00:00:00,31041,Casual,3.0,0.0
2,2018-01-01,00:00:00,31041,Member,1.0,0.0
3,2018-01-01,00:00:00,31042,Member,5.0,0.0
4,2018-01-01,00:00:00,31101,Member,1.0,1.0


In [31]:
final['date'] = final['date'].apply(lambda x:x.strftime('%Y-%m-%d'))

In [32]:
final_data = pd.merge(final, weather_data, on = ['date', 'time'], how = 'inner')

In [33]:
final_data.shape

(2004020, 18)

In [34]:
final_data.columns.tolist()

['date',
 'time',
 'station',
 'member_type',
 'total_out',
 'total_in',
 'weather_description',
 'temp_in_f',
 'humidity_in_%',
 'pressure',
 'visibility_in_miles',
 'wind_direction',
 'wind_speed_in_mph',
 'temp_in_f_delta',
 'pressure_delta',
 'humidity_delta',
 'visibility_delta',
 'wind_speed_delta']

In [35]:
final_data = final_data.fillna('No Description')
final_data.isna().sum()

date                   0
time                   0
station                0
member_type            0
total_out              0
total_in               0
weather_description    0
temp_in_f              0
humidity_in_%          0
pressure               0
visibility_in_miles    0
wind_direction         0
wind_speed_in_mph      0
temp_in_f_delta        0
pressure_delta         0
humidity_delta         0
visibility_delta       0
wind_speed_delta       0
dtype: int64

In [36]:
# write out data file 
final_data.to_csv('./data/data.csv', index=False)